In [ ]:
import os
import random
import numpy as np
import pandas as pd
import geopandas as gpd
from skimage import io
from shapely.geometry import Polygon

import torch
from torch.utils.data import Dataset
from pycocotools.coco import COCO

from dataset_preparing import get_coords_from_densifing_points, generate_heatmap
# dense에 대한 간격이나 표준편차를 하이퍼 파라미터로 조정가능한 코드. 히트맵 생성 때문에 속도는 좀 걸릴 수 있음
# Inria 데이터 크기 조정하여 coco 포맷으로 맞춰준 데이터 처리가능

def min_max_normalize(image, percentile, nodata=-1.):
    image = image.astype('float32')
    mask = np.mean(image, axis=2) != nodata * image.shape[2]

    percent_min = np.percentile(image, percentile, axis=(0, 1))
    percent_max = np.percentile(image, 100-percentile, axis=(0, 1))

    if image.shape[1] * image.shape[0] - np.sum(mask) > 0:
        mdata = np.ma.masked_equal(image, nodata, copy=False)
        mdata = np.ma.filled(mdata, np.nan)
        percent_min = np.nanpercentile(mdata, percentile, axis=(0, 1))

    norm = (image-percent_min) / (percent_max - percent_min)
    norm[norm < 0] = 0
    norm[norm > 1] = 1
    norm = (norm * 255).astype('uint8') * mask[:, :, np.newaxis]

    return norm


def image_graph_collate_road_network_coco(batch):
    images = torch.stack([item['image'] for item in batch], 0).contiguous()
    heatmap = torch.stack([item['heatmap'] for item in batch], 0).contiguous()
    points = [item['nodes'] for item in batch]
    edges = [item['edges'] for item in batch]

    return [images, heatmap, points, edges]


def create_polygon(segmentation):
    # COCO segmentation format is [x1, y1, x2, y2, ..., xn, yn]
    # We need to reshape it to [(x1, y1), (x2, y2), ..., (xn, yn)]
    points = list(zip(segmentation[::2], segmentation[1::2]))
    return Polygon(points)


def gdf_to_nodes_and_edges(gdf):
    nodes = []
    for _, row in gdf.iterrows():
        polygon = row['geometry']
        if polygon.geom_type == 'Polygon':
            for x, y in polygon.exterior.coords:
                nodes.append((x, y))
        elif polygon.geom_type == 'MultiPolygon':
            for part in polygon:
                for x, y in part.exterior.coords:
                    nodes.append((x, y))
        else:
            raise AttributeError

    # Remove duplicates if necessary
    nodes = list(set(nodes))

    # Create a DataFrame for nodes with unique indices
    node_df = pd.DataFrame(nodes, columns=['x', 'y'])
    node_df['node_id'] = range(len(node_df))

    edges = []
    for _, row in gdf.iterrows():
        polygon = row['geometry']
        if polygon.geom_type == 'Polygon':
            coords = polygon.exterior.coords[:-1]  # Exclude closing vertex
            edge = [(node_df[(node_df['x'] == x) & (node_df['y'] == y)].index[0], 
                    node_df[(node_df['x'] == coords[(i+1)%len(coords)][0]) & (node_df['y'] == coords[(i+1)%len(coords)][1])].index[0]) 
                    for i, (x, y) in enumerate(coords)]
            edges.extend(edge)
        elif polygon.geom_type == 'MultiPolygon':
            for part in polygon:
                coords = part.exterior.coords[:-1]
                edge = [(node_df[(node_df['x'] == x) & (node_df['y'] == y)].index[0], 
                        node_df[(node_df['x'] == coords[(i+1)%len(coords)][0]) & (node_df['y'] == coords[(i+1)%len(coords)][1])].index[0]) 
                        for i, (x, y) in enumerate(coords)]
                edges.extend(edge)

    return node_df[['y', 'x']].values, edges


class CrowdAI(Dataset):
    """A dataset class for handling and processing data from the CrowdAI dataset.

    Attributes:
        IMAGES_DIRECTORY (str): Directory containing the images.
        ANNOTATIONS_PATH (str): File path for the annotations.
        coco (COCO): COCO object to handle COCO annotations.
        max_points (int): Maximum number of points to consider (default 256).
        gap_distance (float): Distance between interpolated points.
        sigma (float): Standard deviation for Gaussian kernel used in heatmap generation.

    Args:
        images_directory (str): Directory where the dataset images are stored.
        annotations_path (str): File path for the COCO format annotations.
        gap_distance (int, optional): Gap distance for densifying points. Defaults to 20.
        sigma (float, optional): Sigma value for Gaussian blur in heatmap. Defaults to 1.5.
    """

    def __init__(self, 
                 images_directory, 
                 annotations_path,
                 gap_datance=20,
                 sigma=1.5):

        self.IMAGES_DIRECTORY = images_directory
        self.ANNOTATIONS_PATH = annotations_path
        self.coco = COCO(self.ANNOTATIONS_PATH)
        self.image_ids = self.coco.getImgIds(catIds=self.coco.getCatIds())

        self.len = len(self.image_ids)

        self.max_points = 256 # TODO: It should be restricted the number when gt points over the max points limit
        self.gap_distance = gap_datance
        self.sigma = sigma

    def prepare_annotations(self, img):
        """Prepares annotations for an image.
        Args:
            img (dict): A dictionary containing image metadata.

        Returns:
            GeoDataFrame: A GeoDataFrame containing the geometrical data of annotations.
        """
        annotation_ids = self.coco.getAnnIds(imgIds=img['id'])
        annotations = self.coco.loadAnns(annotation_ids)
        random.shuffle(annotations)

        data = []
        for ann in annotations:
            polygon = create_polygon(ann['segmentation'][0])
            data.append({'id': ann['id'], 'geometry': polygon})
        gdf = gpd.GeoDataFrame(data, geometry='geometry')
        return gdf

    def loadSample(self, idx):
        """Loads a sample for a given index.

        Args:
            idx (int): The index of the sample to load.

        Returns:
            dict: A dictionary containing the sample data.
                'image' (torch.Tensor of shape [3, H, W], torch.float32): 
                    The image tensor normalized to [0, 1].
                'image_idx' (torch.Tensor of shape [1], torch.long): 
                    The index of the image.
                'heatmap' (torch.Tensor of shape [H, W], torch.float32): 
                    The heatmap tensor for the image normalized to [0, 1]. 
                'nodes' (torch.Tensor of shape [N, 2], torch.float): 
                    The nodes tensor representing points in the image.
                    nodes are normalized to [0, 1]
                'edges' (torch.Tensor of shape [E, 2], torch.long): 
                    The edges tensor representing connections between nodes.
        """
        idx = self.image_ids[idx]

        img = self.coco.loadImgs(idx)[0]
        image_path = os.path.join(self.IMAGES_DIRECTORY, img['file_name'])
        image = io.imread(image_path)

        gdf = self.prepare_annotations(img)
        coords, gdf = get_coords_from_densifing_points(gdf, gap_distance=self.gap_distance) # [N, 2]
        heatmap = generate_heatmap(coords, image.shape[:2], sigma=self.sigma)

        nodes, edges = gdf_to_nodes_and_edges(gdf)
        nodes = nodes / image.shape[0]

        image_idx = torch.tensor([idx])
        image = torch.from_numpy(image)
        image = image.permute(2,0,1) / 255.0
        heatmap = torch.from_numpy(heatmap) / 255.0
        
        nodes = torch.tensor(nodes, dtype=torch.float)
        edges = torch.tensor(edges, dtype=torch.long)

        sample = {
            'image': image, 
            'image_idx': image_idx, 
            'heatmap': heatmap,
            'nodes': nodes,
            'edges': edges
            }
        return sample

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        sample = self.loadSample(idx)
        return sample

In [ ]:
dataset = CrowdAI(images_directory='/nas/tsgil/dataset/Inria_building/cocostyle/images',
                    annotations_path='/nas/tsgil/dataset/Inria_building/cocostyle/annotation.json')
dataloader = torch.utils.data.DataLoader(dataset, batch_size=6, collate_fn=image_graph_collate_road_network_coco)

print(next(iter(dataloader))[0].shape) # image

data = next(iter(dataloader))

image = data[0][1].detach().cpu().numpy().transpose(1,2,0)
heatmap =  data[1][1].detach().cpu().numpy()
nodes = data[2][1].detach().cpu().numpy() * image.shape[0]
edges = data[3][1].detach().cpu().numpy()

nodes = nodes.astype('int64')

# Visualize
import matplotlib.pyplot as plt
plt.imshow(min_max_normalize(image, 0))
plt.scatter(nodes[:,1], nodes[:,0], color='r')

for e in edges:
    connect = np.stack([nodes[e[0]], nodes[e[1]]], axis=0)
    plt.plot(connect[:,1], connect[:,0])
plt.show()

### 임의의 사각형 N 타겟 노드와 엣지 리턴 코드

In [ ]:
def generate_directed_adjacency_matrix(pairs_list):
    """ 주어진 노드 쌍 리스트를 기반으로 방향성 인접 행렬을 생성합니다. """
    import numpy as np

    # 인접 행렬 크기 계산 (최대 노드 인덱스 + 1)
    max_index = max(max(pair) for pair in pairs_list)
    adjacency_matrix = np.zeros((max_index + 1, max_index + 1), dtype=int)

    # 인접 행렬 채우기
    for i, j in pairs_list:
        adjacency_matrix[i, j] = 1  # 방향성 그래프: i에서 j로의 방향

    return adjacency_matrix

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd

from shapely.geometry import Polygon

from dataset_preparing import get_coords_from_densifing_points
from dataloader_cocostyle import gdf_to_nodes_and_edges

# shapes and dtypes of nodes/edges are exactly same as what I defined in InriaDataLoader

original_polygon = Polygon([(0, 0), (100, 0), (100, 100), (0, 100)])
# original_polygon = Polygon([(10, 10), (90, 10), (90, 90), (10, 90)])
gdf = gpd.GeoDataFrame(geometry = [original_polygon])

coords, gdf = get_coords_from_densifing_points(gdf, gap_distance=25)
nodes, edges = gdf_to_nodes_and_edges(gdf)
nodes = nodes / 100

nodes = nodes.astype('float32')
edges = np.array(edges).astype('int64')


# Visualizer
plt.scatter(nodes[:,1], nodes[:,0])
for e in edges:
    plt.plot(nodes[e,1], nodes[e,0])

plt.show()
generate_directed_adjacency_matrix(edges)
print(nodes)

In [ ]:
from models.DGS import HungarianMatcher
import torch

N = len(nodes)
K = 50

matcher = HungarianMatcher()
matcher.eval()
output = { # K개의 샘플링 뽑는 걸로 수정하기
    "pred_nodes": torch.rand(1, K, 2),
    } # x_cord, y_cord
target = {'nodes':[torch.tensor(nodes)]}
out = matcher(output, target)
print(out[0][0])
print(out[0][1])
mapping = {j:i for i,j in zip(out[0][0].tolist(),out[0][1].tolist())}
print(mapping)
gt = [nodes[node] for node in out[0][1]]
sample = output['pred_nodes'][0]
print(gt)
print(sample)
print(nodes[0],nodes[9])
print(edges)
plt.scatter(sample[:,1], sample[:,0])
if K<N:
    mapping2 = {edge[0]: edge[1] for edge in edges}
    sample_edge = []
    for i, j in edges:
        if i not in mapping:
            while i not in mapping:
                i = mapping2[i]
        if j not in mapping:
            while j not in mapping:
                j = mapping2[j]
        if mapping[i] != mapping[j]:
            sample_edge.append([mapping[i],mapping[j]])
else:
    sample_edge = [[mapping[i],mapping[j]] for i, j in edges]
for e in sample_edge:
    plt.plot(sample[e,1], sample[e,0])
plt.show()
print(generate_directed_adjacency_matrix(sample_edge))

In [ ]:
sample_edge

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import geopandas as gpd
from skimage import io
from shapely.geometry import Polygon

import torch
from torch.utils.data import Dataset
from pycocotools.coco import COCO

from dataset_preparing import get_coords_from_densifing_points, generate_heatmap
# dense에 대한 간격이나 표준편차를 하이퍼 파라미터로 조정가능한 코드. 히트맵 생성 때문에 속도는 좀 걸릴 수 있음
# Inria 데이터 크기 조정하여 coco 포맷으로 맞춰준 데이터 처리가능

def min_max_normalize(image, percentile, nodata=-1.):
    image = image.astype('float32')
    mask = np.mean(image, axis=2) != nodata * image.shape[2]

    percent_min = np.percentile(image, percentile, axis=(0, 1))
    percent_max = np.percentile(image, 100-percentile, axis=(0, 1))

    if image.shape[1] * image.shape[0] - np.sum(mask) > 0:
        mdata = np.ma.masked_equal(image, nodata, copy=False)
        mdata = np.ma.filled(mdata, np.nan)
        percent_min = np.nanpercentile(mdata, percentile, axis=(0, 1))

    norm = (image-percent_min) / (percent_max - percent_min)
    norm[norm < 0] = 0
    norm[norm > 1] = 1
    norm = (norm * 255).astype('uint8') * mask[:, :, np.newaxis]

    return norm


def image_graph_collate_road_network_coco(batch):
    images = torch.stack([item['image'] for item in batch], 0).contiguous()
    heatmap = torch.stack([item['heatmap'] for item in batch], 0).contiguous()
    points = [item['nodes'] for item in batch]
    edges = [item['edges'] for item in batch]

    return [images, heatmap, points, edges]


def create_polygon(segmentation):
    # COCO segmentation format is [x1, y1, x2, y2, ..., xn, yn]
    # We need to reshape it to [(x1, y1), (x2, y2), ..., (xn, yn)]
    points = list(zip(segmentation[::2], segmentation[1::2]))
    return Polygon(points)


def gdf_to_nodes_and_edges(gdf):
    nodes = []
    for _, row in gdf.iterrows():
        polygon = row['geometry']
        if polygon.geom_type == 'Polygon':
            for x, y in polygon.exterior.coords:
                nodes.append((x, y))
        elif polygon.geom_type == 'MultiPolygon':
            for part in polygon:
                for x, y in part.exterior.coords:
                    nodes.append((x, y))
        else:
            raise AttributeError

    # Remove duplicates if necessary
    nodes = list(set(nodes))

    # Create a DataFrame for nodes with unique indices
    node_df = pd.DataFrame(nodes, columns=['x', 'y'])
    node_df['node_id'] = range(len(node_df))

    edges = []
    for _, row in gdf.iterrows():
        polygon = row['geometry']
        if polygon.geom_type == 'Polygon':
            coords = polygon.exterior.coords[:-1]  # Exclude closing vertex
            edge = [(node_df[(node_df['x'] == x) & (node_df['y'] == y)].index[0], 
                    node_df[(node_df['x'] == coords[(i+1)%len(coords)][0]) & (node_df['y'] == coords[(i+1)%len(coords)][1])].index[0]) 
                    for i, (x, y) in enumerate(coords)]
            edges.extend(edge)
        elif polygon.geom_type == 'MultiPolygon':
            for part in polygon:
                coords = part.exterior.coords[:-1]
                edge = [(node_df[(node_df['x'] == x) & (node_df['y'] == y)].index[0], 
                        node_df[(node_df['x'] == coords[(i+1)%len(coords)][0]) & (node_df['y'] == coords[(i+1)%len(coords)][1])].index[0]) 
                        for i, (x, y) in enumerate(coords)]
                edges.extend(edge)

    return node_df[['y', 'x']].values, edges


class CrowdAI(Dataset):
    """A dataset class for handling and processing data from the CrowdAI dataset.

    Attributes:
        IMAGES_DIRECTORY (str): Directory containing the images.
        ANNOTATIONS_PATH (str): File path for the annotations.
        coco (COCO): COCO object to handle COCO annotations.
        max_points (int): Maximum number of points to consider (default 256).
        gap_distance (float): Distance between interpolated points.
        sigma (float): Standard deviation for Gaussian kernel used in heatmap generation.

    Args:
        images_directory (str): Directory where the dataset images are stored.
        annotations_path (str): File path for the COCO format annotations.
        gap_distance (int, optional): Gap distance for densifying points. Defaults to 20.
        sigma (float, optional): Sigma value for Gaussian blur in heatmap. Defaults to 1.5.
    """

    def __init__(self, 
                 images_directory, 
                 annotations_path,
                 gap_datance=20,
                 sigma=1.5):

        self.IMAGES_DIRECTORY = images_directory
        self.ANNOTATIONS_PATH = annotations_path
        self.coco = COCO(self.ANNOTATIONS_PATH)
        self.image_ids = self.coco.getImgIds(catIds=self.coco.getCatIds())

        self.len = len(self.image_ids)

        self.max_points = 256 # TODO: It should be restricted the number when gt points over the max points limit
        self.gap_distance = gap_datance
        self.sigma = sigma

    def prepare_annotations(self, img):
        """Prepares annotations for an image.
        Args:
            img (dict): A dictionary containing image metadata.

        Returns:
            GeoDataFrame: A GeoDataFrame containing the geometrical data of annotations.
        """
        annotation_ids = self.coco.getAnnIds(imgIds=img['id'])
        annotations = self.coco.loadAnns(annotation_ids)
        random.shuffle(annotations)

        data = []
        for ann in annotations:
            polygon = create_polygon(ann['segmentation'][0])
            data.append({'id': ann['id'], 'geometry': polygon})
        gdf = gpd.GeoDataFrame(data, geometry='geometry')
        return gdf

    def loadSample(self, idx):
        """Loads a sample for a given index.

        Args:
            idx (int): The index of the sample to load.

        Returns:
            dict: A dictionary containing the sample data.
                'image' (torch.Tensor of shape [3, H, W], torch.float32): 
                    The image tensor normalized to [0, 1].
                'image_idx' (torch.Tensor of shape [1], torch.long): 
                    The index of the image.
                'heatmap' (torch.Tensor of shape [H, W], torch.float32): 
                    The heatmap tensor for the image normalized to [0, 1]. 
                'nodes' (torch.Tensor of shape [N, 2], torch.float): 
                    The nodes tensor representing points in the image.
                    nodes are normalized to [0, 1]
                'edges' (torch.Tensor of shape [E, 2], torch.long): 
                    The edges tensor representing connections between nodes.
        """
        idx = self.image_ids[idx]

        img = self.coco.loadImgs(idx)[0]
        image_path = os.path.join(self.IMAGES_DIRECTORY, img['file_name'])
        image = io.imread(image_path)

        gdf = self.prepare_annotations(img)
        coords, gdf = get_coords_from_densifing_points(gdf, gap_distance=self.gap_distance) # [N, 2]
        heatmap = generate_heatmap(coords, image.shape[:2], sigma=self.sigma)

        nodes, edges = gdf_to_nodes_and_edges(gdf)
        nodes = nodes / image.shape[0]

        image_idx = torch.tensor([idx])
        image = torch.from_numpy(image)
        image = image.permute(2,0,1) / 255.0
        heatmap = torch.from_numpy(heatmap) / 255.0
        
        nodes = torch.tensor(nodes, dtype=torch.float)
        edges = torch.tensor(edges, dtype=torch.long)

        sample = {
            'image': image, 
            'image_idx': image_idx, 
            'heatmap': heatmap,
            'nodes': nodes,
            'edges': edges
            }
        return sample

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        sample = self.loadSample(idx)
        return sample

In [ ]:
dataset = CrowdAI(images_directory='/nas/tsgil/dataset/Inria_building/cocostyle/images',
                    annotations_path='/nas/tsgil/dataset/Inria_building/cocostyle/annotation.json')
dataloader = torch.utils.data.DataLoader(dataset, batch_size=6, collate_fn=image_graph_collate_road_network_coco)

print(next(iter(dataloader))[0].shape) # image

data = next(iter(dataloader))

image = data[0][1].detach().cpu().numpy().transpose(1,2,0)
heatmap =  data[1][1].detach().cpu().numpy()
nodes = data[2][1].detach().cpu().numpy() * image.shape[0]
edges = data[3][1].detach().cpu().numpy()

nodes = nodes.astype('int64')

# Visualize
import matplotlib.pyplot as plt
plt.imshow(min_max_normalize(image, 0))
plt.scatter(nodes[:,1], nodes[:,0], color='r')

for e in edges:
    connect = np.stack([nodes[e[0]], nodes[e[1]]], axis=0)
    plt.plot(connect[:,1], connect[:,0])
plt.show()

In [ ]:
import geopandas as gpd

gdf = gpd.GeoDataFrame({'k':[1, None]})

In [ ]:
import torch.nn as nn

# Example model
model = nn.Sequential(
    nn.Linear(10, 20), # 10->20, 파라미터 20*10개
    nn.ReLU(),
    nn.Linear(20, 10) # 20->10, 파라미터 10*20개
)

# Print the weights of the first linear layer
for name, param in model.named_parameters():
    if name == '0.weight':  # '0' refers to the first layer
        print(param)
a = model.state_dict()
print(a.keys())
a['2.weight'].shape

In [6]:
import torch
from models.TopDiG import build_TopDiG
from models.backbone_R2U_Net import R2U_Net, R2U_Net_origin
ck_TopDiG = torch.load('/nas/tsgil/relationformer/work_dirs/TopDiG_train/runs/baseline_TopDiG_train_epoch8_10/models/epochs_8.pth', map_location='cpu')
ck_backbone = torch.load('/nas/tsgil/relationformer/work_dirs/R2U_Net_pretrain/runs/baseline_R2U_Net_pretrain_epoch200_10/models/epochs_200.pth', map_location='cpu')

In [9]:
print(ck_backbone.keys())
print(ck_backbone['model_state_dict'].keys())
print(ck_backbone['model_state_dict']['RRCNN1.RCNN.0.conv.0.weight'])

model1 = build_TopDiG('/nas/tsgil/relationformer/configs/TopDiG_train.yaml')
model2 = R2U_Net()
model3 = R2U_Net_origin()
model1_dict = model1.state_dict()
print(model1_dict['model_state_dict']['encoder.RRCNN1.RCNN.0.conv.0.weight'])
model1.load_state_dict(ck_backbone['model_state_dict'], strict=False)
total_params = sum(p.numel() for p in model1.parameters())
model3_dict = model3.state_dict()
model2_dict = model2.state_dict()
model1_dict = model1.state_dict()
print(model1_dict['model_state_dict']['encoder.RRCNN1.RCNN.0.conv.0.weight'])

dict_keys(['epoch', 'model_state_dict', 'optimizer_state_dict', 'schedulaer_state_dict'])
odict_keys(['RRCNN1.RCNN.0.conv.0.weight', 'RRCNN1.RCNN.0.conv.0.bias', 'RRCNN1.RCNN.0.conv.1.weight', 'RRCNN1.RCNN.0.conv.1.bias', 'RRCNN1.RCNN.0.conv.1.running_mean', 'RRCNN1.RCNN.0.conv.1.running_var', 'RRCNN1.RCNN.0.conv.1.num_batches_tracked', 'RRCNN1.RCNN.1.conv.0.weight', 'RRCNN1.RCNN.1.conv.0.bias', 'RRCNN1.RCNN.1.conv.1.weight', 'RRCNN1.RCNN.1.conv.1.bias', 'RRCNN1.RCNN.1.conv.1.running_mean', 'RRCNN1.RCNN.1.conv.1.running_var', 'RRCNN1.RCNN.1.conv.1.num_batches_tracked', 'RRCNN1.Conv_1x1.weight', 'RRCNN1.Conv_1x1.bias', 'RRCNN2.RCNN.0.conv.0.weight', 'RRCNN2.RCNN.0.conv.0.bias', 'RRCNN2.RCNN.0.conv.1.weight', 'RRCNN2.RCNN.0.conv.1.bias', 'RRCNN2.RCNN.0.conv.1.running_mean', 'RRCNN2.RCNN.0.conv.1.running_var', 'RRCNN2.RCNN.0.conv.1.num_batches_tracked', 'RRCNN2.RCNN.1.conv.0.weight', 'RRCNN2.RCNN.1.conv.0.bias', 'RRCNN2.RCNN.1.conv.1.weight', 'RRCNN2.RCNN.1.conv.1.bias', 'RRCNN2.RCNN.1.co

KeyError: 'model_state_dict'

In [4]:
print(len(model3_dict.keys()), model3_dict.keys())
print(len(model2_dict.keys()), model2_dict.keys())
print(len(model1_dict.keys()), model1_dict.keys())

172 odict_keys(['RRCNN1.RCNN.0.conv.0.weight', 'RRCNN1.RCNN.0.conv.0.bias', 'RRCNN1.RCNN.0.conv.1.weight', 'RRCNN1.RCNN.0.conv.1.bias', 'RRCNN1.RCNN.0.conv.1.running_mean', 'RRCNN1.RCNN.0.conv.1.running_var', 'RRCNN1.RCNN.0.conv.1.num_batches_tracked', 'RRCNN1.RCNN.1.conv.0.weight', 'RRCNN1.RCNN.1.conv.0.bias', 'RRCNN1.RCNN.1.conv.1.weight', 'RRCNN1.RCNN.1.conv.1.bias', 'RRCNN1.RCNN.1.conv.1.running_mean', 'RRCNN1.RCNN.1.conv.1.running_var', 'RRCNN1.RCNN.1.conv.1.num_batches_tracked', 'RRCNN1.Conv_1x1.weight', 'RRCNN1.Conv_1x1.bias', 'RRCNN2.RCNN.0.conv.0.weight', 'RRCNN2.RCNN.0.conv.0.bias', 'RRCNN2.RCNN.0.conv.1.weight', 'RRCNN2.RCNN.0.conv.1.bias', 'RRCNN2.RCNN.0.conv.1.running_mean', 'RRCNN2.RCNN.0.conv.1.running_var', 'RRCNN2.RCNN.0.conv.1.num_batches_tracked', 'RRCNN2.RCNN.1.conv.0.weight', 'RRCNN2.RCNN.1.conv.0.bias', 'RRCNN2.RCNN.1.conv.1.weight', 'RRCNN2.RCNN.1.conv.1.bias', 'RRCNN2.RCNN.1.conv.1.running_mean', 'RRCNN2.RCNN.1.conv.1.running_var', 'RRCNN2.RCNN.1.conv.1.num_batc